# Multiple Choice Answer Type

This notebook demonstrates how to generate multiple choice questions with predefined answer options. Multiple choice questions are perfect for educational content, knowledge testing, and structured predictions with a fixed set of choices.

In [1]:
%pip install lightningrod-ai==0.1.5

from IPython.display import clear_output
clear_output()

## Set up the client

**Set your API key:**

Go to the Secrets section (🔑 icon in left sidebar) and add a secret named `LIGHTNINGROD_API_KEY`

In [2]:
from google.colab import userdata
from lightningrod import LightningRod

api_key = userdata.get("LIGHTNINGROD_API_KEY")
lr = LightningRod(api_key=api_key)

## Multiple Choice Answer Type Overview

Multiple choice questions provide a question with several predefined answer options (typically 2-5 choices). The `QuestionAndLabelGenerator` generates both the question and the answer options together, then determines which option is correct.

Ideal for:
- Educational content and quizzes
- Knowledge testing
- Structured predictions with fixed choices
- Scenarios where you want to limit possible answers

In [3]:
from datetime import datetime, timedelta
from lightningrod import AnswerType, AnswerTypeEnum, NewsSeedGenerator, QuestionGenerator, QuestionRenderer, QuestionPipeline, WebSearchLabeler

answer_type = AnswerType(answer_type=AnswerTypeEnum.MULTIPLE_CHOICE)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    search_query="technology industry news",
)

question_generator = QuestionGenerator(
    instructions=(
        "Generate multiple choice questions about technology industry events. "
        "Each question should have 3-4 answer options, with one clearly correct answer."
    ),
    examples=[
        "Question: Which company will release a new AI model first in 2025?\n"
        "A) OpenAI\n"
        "B) Google\n"
        "C) Anthropic\n"
        "Label: A",
        "Question: What will be the main focus of Apple's next product launch?\n"
        "A) AI features\n"
        "B) Battery life\n"
        "C) Camera improvements\n"
        "D) Design changes\n"
        "Label: A",
    ],
    bad_examples=[
        "Question: Will AI be important? Label: Yes",
        "Question: What is AI? Label: Artificial Intelligence",
    ],
    questions_per_seed=2,
    answer_type=answer_type,
)

# Labeler automatically finds answers to questions using web search
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

# Renderer formats the question output
renderer = QuestionRenderer(answer_type=answer_type)

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

## Run the Pipeline

In [4]:
dataset = lr.transforms.run(pipeline_config, max_questions=10) # keep max questions low when testing

> Note: This can take a few minutes to complete processing.

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [5]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [6]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
df.head()

Generated 20 samples



,question.question_text,label.label,label.label_confidence,label.resolution_date,label.reasoning,label.answer_sources,prompt,seed.seed_text,seed.url,seed.seed_creation_date,seed.search_query,is_valid,meta.sample_id,meta.parent_sample_id,meta.processing_time_ms,meta.filter_reason
0,"Question: According to the article, what is a ...",A,1.0,2026-01-13T00:00:00,Multiple articles indicate that a primary reas...,https://vertexaisearch.cloud.google.com/ground...,"QUESTION:\nQuestion: According to the article,...",Title: Why SF isn't seeing job surge amid AI b...,https://www.sfexaminer.com/news/technology/san...,2026-01-14T00:00:00,technology industry news,True,c591df88-6dd7-4ad9-9dfb-4f56e2a2a058,0ff4896e-ad9e-40b3-b9b5-f50b20a996dd,13962.545,NaN
1,Question: Which organization announced at CES ...,D,1.0,2026-01-05T00:00:00,Boston Dynamics announced at CES 2026 a partne...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nQuestion: Which organization announ...,Title: Humanoid robots take over CES in Las Ve...,https://www.cnbc.com/2026/01/09/humanoid-robot...,2026-01-07T00:00:00,technology industry news,True,27a0f8fe-b915-4e53-b86f-97c827872de0,d915c83f-6138-49d7-bb2c-306effa0939f,7915.373,NaN
2,Question: Which sector has recently dominated ...,A,1.0,2025-10-26T00:00:00,Multiple recent reports and analyses of techno...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nQuestion: Which sector has recently...,Title: Just a moment...\n\nURL Source: https:/...,https://pitchbook.com/news/articles/unicorn-st...,2026-01-07T00:00:00,technology industry news,True,a0639b40-c8d2-4f73-a354-0cfa21c94952,51cc27ac-85c8-40b3-b97b-cb1905c3360e,12608.989,NaN
3,Question: Based on the metadata provided in th...,Undetermined,1.0,NaN,The question asks to determine the publication...,NaN,NaN,Title: CES 2026: What to expect from the tech ...,https://finance.yahoo.com/news/ces-2026-what-t...,2026-01-07T00:00:00,technology industry news,False,365e63f6-7520-4713-a782-308a45edb794,fabb5d0d-0921-48ad-acb6-2c1bdfbc5acd,3361.403,Undetermined label
4,Question: According to the January 2026 announ...,B,1.0,2026-01-06T00:00:00,According to multiple news sources reporting o...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nQuestion: According to the January ...,Title: Siemens and NVIDIA Expand Partnership t...,https://nvidianews.nvidia.com/news/siemens-and...,2026-01-07T00:00:00,technology industry news,True,c2d9ac08-004d-489f-bb65-071fc0d3b6fc,d4a3c9dd-f35c-4753-ad91-3ad2e4013928,7085.279,NaN


## Use Cases for Multiple Choice Questions

**Educational Content:**
- Quiz questions for learning materials
- Knowledge assessment questions
- Study guides

**Structured Predictions:**
- "Which team will win?" with team options
- "What will be the outcome?" with predefined scenarios
- "Which product will launch first?" with product options